In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
# from actor import ActorNetwork
# from rbfActor import RbfActorNetwork
# from critic import CriticNetwork
from actorCmaRbf import ActorCmaRbfNetwork 
# from replay_buffer import ReplayBuffer
# from ounoise import OUNoise
import gym, time, cma
from Envs.reaching import ReachingEnv
# from Envs.throwing import ThrowingEnv
import matplotlib.pyplot as plt

In [2]:
MAX_EPISODE = 1000
MAX_TIME = 200
RANDOM_SEED = 1926

goal_pos = np.load('./Envs/reaching_goal_pos.npy')

env = ReachingEnv(include_t = True)
# env = ThrowingEnv(include_t = True)
tf.set_random_seed(RANDOM_SEED)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high
sess = tf.Session()

total_length = 51*2
hidden_layer_dim = [40,30]
# if len(hidden_layer_dim) == 2:
#     bound1 = state_dim * hidden_layer_dim[0]
#     bound2 = bound1 + hidden_layer_dim[0]*hidden_layer_dim[1]
#     bound3 = bound2 + hidden_layer_dim[1]*action_dim
#     bound4 = bound3 + hidden_layer_dim[0]
#     bound5 = bound4 + hidden_layer_dim[1]
#     total_length = bound5
# elif len(hidden_layer_dim) == 1:
#     bound1 = state_dim * hidden_layer_dim[0]
#     bound2 = bound1 + hidden_layer_dim[0]*action_dim
#     bound3 = bound2 + hidden_layer_dim[0]
#     total_length = bound3

In [3]:
actor = ActorCmaRbfNetwork(sess, state_dim, action_dim, action_bound, hidden_layer_dim = hidden_layer_dim, seed = RANDOM_SEED)

In [4]:
def cost_fcn_cma(network_vec, env):
    state = env.reset()
    net_dict = wrap_network(network_vec, len(hidden_layer_dim))
    reward_list = []
    for t in range(MAX_TIME):
        action = actor.predict(np.reshape(state, (-1, state_dim)), net_dict)
        action = np.clip(action, -action_bound, action_bound)
        action = np.reshape(action, action_dim)
        next_state, reward, done, info = env.step(action)
        reward_list.append(reward)
        if done:
            break
        state = next_state
#         if t%100 == 0:
# #             print(action)
    return -np.sum(reward_list)

In [5]:
def wrap_network(network_vector, net_depth = 2):
    
    net_dict = {}
    net_dict['weights1'] = np.reshape(network_vector, [51, 2])
#     if net_depth == 2:
#         bound1 = state_dim*hidden_layer_dim[0]
#         bound2 = bound1 + hidden_layer_dim[0]*hidden_layer_dim[1]
#         bound3 = bound2 + hidden_layer_dim[1]*action_dim
#         bound4 = bound3 + hidden_layer_dim[0]
#         bound5 = bound4 + hidden_layer_dim[1]
#         net_dict['weights1'] = np.reshape(network_vector[0:bound1], [state_dim, hidden_layer_dim[0]])
#         net_dict['weights2'] = np.reshape(network_vector[bound1:bound2], [hidden_layer_dim[0], hidden_layer_dim[1]])
#         net_dict['weights3'] = np.reshape(network_vector[bound2:bound3], [hidden_layer_dim[1], action_dim])
#         net_dict['bias1'] = np.reshape(network_vector[bound3:bound4], [hidden_layer_dim[0]])
#         net_dict['bias2'] = np.reshape(network_vector[bound4:bound5], [hidden_layer_dim[1]])
#     elif net_depth == 1:
#         bound1 = state_dim * hidden_layer_dim[0]
#         bound2 = bound1 + hidden_layer_dim[0]*action_dim
#         bound3 = bound2 + hidden_layer_dim[0]
#         net_dict['weights1'] = np.reshape(network_vector[0:bound1], [state_dim, hidden_layer_dim[0]])
#         net_dict['weights2'] = np.reshape(network_vector[bound1:bound2], [hidden_layer_dim[0], action_dim])
#         net_dict['bias1'] = np.reshape(network_vector[bound2:bound3], [hidden_layer_dim[0]])
    return net_dict



In [6]:
for i in range(50):
    init_policy_vec = np.random.normal(0., 1., total_length)
    # init_policy_vec[bound2:bound3] /= 333. 
    # sigma0 = np.ones(total_length)
    # sigma0[bound1:bound2] /= 33.
    sigma0 = 1.
    opts = cma.CMAOptions()
    opts.set('verb_filenameprefix','Data/reaching_cma_rbf_50/'+str(i))
    es = cma.CMAEvolutionStrategy(init_policy_vec,sigma0,opts)
    objfun = lambda network_vec: cost_fcn_cma(network_vec, env)
    logger = cma.CMADataLogger().register(es)
    es.optimize(objfun, verb_disp = True, iterations = 50)

/usr/local/lib/python2.7/dist-packages/cma.py:2938: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if x0 == str(x0):


(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 102 (seed=1011323, Mon Jun 19 11:47:45 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.643429423838708e+02 1.0e+00 9.60e-01  1e+00  1e+00 0:7.5
    2     34 1.454129214228745e+02 1.0e+00 9.30e-01  9e-01  9e-01 0:14.0
    3     51 1.561380209436848e+02 1.0e+00 9.06e-01  9e-01  9e-01 0:21.0
    4     68 1.577991805255903e+02 1.0e+00 8.88e-01  9e-01  9e-01 0:28.1
    5     85 1.628398047562050e+02 1.0e+00 8.71e-01  9e-01  9e-01 0:35.3
    6    102 1.535621346932058e+02 1.0e+00 8.57e-01  9e-01  9e-01 0:43.0
    7    119 1.334944047738945e+02 1.1e+00 8.48e-01  8e-01  9e-01 0:49.9
    8    136 1.527509686663124e+02 1.1e+00 8.39e-01  8e-01  8e-01 0:56.9
    9    153 1.522755613282711e+02 1.1e+00 8.33e-01  8e-01  8e-01 1:4.5
   10    170 1.258570291596032e+02 1.1e+00 8.24e-01  8e-01  8e-01 1:11.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
   11    187 1.236293464903924e+02